# Pré-processamento de Dados para Análise de Noisy Neighbors em Kubernetes

Este notebook demonstra o processo de carregamento, limpeza e pré-processamento dos dados coletados durante o experimento de detecção de "noisy neighbors" em clusters Kubernetes.

## Objetivos
- Carregar dados de métricas dos diferentes tenants e fases
- Limpar e normalizar os dados 
- Converter timestamps para tempo decorrido
- Preparar datasets consolidados para análise

## Configuração Inicial

Primeiro, vamos importar as bibliotecas necessárias e configurar o ambiente para análise.

In [ ]:
# Importação das bibliotecas
import os
import glob
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import re

# Configuração para visualizações de qualidade acadêmica
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (10, 6)
plt.rcParams['figure.dpi'] = 300
plt.rcParams['font.size'] = 12
plt.rcParams['axes.labelsize'] = 14
plt.rcParams['axes.titlesize'] = 16
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12
plt.rcParams['legend.fontsize'] = 12
plt.rcParams['legend.title_fontsize'] = 14

# Caminho base para os dados do experimento
BASE_DATA_DIR = "/home/phil/Projects/k8s-noisy-detection/demo-data/demo-experiment-3-rounds"

## Funções de Carregamento de Dados

Vamos criar funções flexíveis que permitam carregar dados de diferentes tenants, fases e métricas conforme a necessidade do usuário.

In [ ]:
def list_available_tenants(experiment_dir):
    """
    Lista todos os tenants disponíveis no diretório do experimento.
    
    Args:
        experiment_dir (str): Caminho para o diretório do experimento
        
    Returns:
        list: Lista de nomes de tenants
    """
    tenants = set()
    
    # Procura por diretórios de tenants em todas as fases e rounds
    for round_dir in glob.glob(os.path.join(experiment_dir, "round-*")):
        for phase_dir in glob.glob(os.path.join(round_dir, "*")):
            # Listamos diretórios dentro de cada fase
            for item in os.listdir(phase_dir):
                item_path = os.path.join(phase_dir, item)
                if os.path.isdir(item_path):
                    tenants.add(item)
    
    return sorted(list(tenants))

def list_available_metrics(experiment_dir, tenant="tenant-a"):
    """
    Lista todas as métricas disponíveis para um tenant específico.
    
    Args:
        experiment_dir (str): Caminho para o diretório do experimento
        tenant (str): Nome do tenant para listar métricas
        
    Returns:
        list: Lista de nomes de métricas
    """
    metrics = set()
    
    # Procura arquivos CSV dentro dos diretórios do tenant especificado
    pattern = os.path.join(experiment_dir, "round-*", "*", tenant, "*.csv")
    for csv_file in glob.glob(pattern):
        metric_name = os.path.basename(csv_file).replace(".csv", "")
        metrics.add(metric_name)
    
    return sorted(list(metrics))

def parse_timestamp(timestamp_str):
    """
    Converte string de timestamp no formato usado nos arquivos CSV para objeto datetime.
    
    Args:
        timestamp_str (str): String de timestamp no formato "YYYYMMDD_HHMMSS"
        
    Returns:
        datetime: Objeto datetime correspondente
    """
    return datetime.strptime(timestamp_str, "%Y%m%d_%H%M%S")

def load_metric_data(experiment_dir, metric_name, tenants=None, phases=None, rounds=None):
    """
    Carrega dados de uma métrica específica para os tenants, fases e rounds selecionados.
    
    Args:
        experiment_dir (str): Caminho para o diretório do experimento
        metric_name (str): Nome da métrica a ser carregada
        tenants (list): Lista de tenants a incluir (None = todos)
        phases (list): Lista de fases a incluir (None = todas)
        rounds (list): Lista de rounds a incluir (None = todos)
        
    Returns:
        DataFrame: DataFrame consolidado com os dados da métrica
    """
    all_data = []
    
    # Se nenhuma lista específica for fornecida, incluir tudo
    if tenants is None:
        tenants = list_available_tenants(experiment_dir)
    if rounds is None:
        rounds = [f"round-{i}" for i in range(1, 10)]  # Assume até 9 rounds
        rounds = [r for r in rounds if os.path.exists(os.path.join(experiment_dir, r))]
    if phases is None:
        phases = ["*"]  # Glob pattern para pegar todas as fases
    
    for round_name in rounds:
        for phase_pattern in phases:
            phase_dirs = glob.glob(os.path.join(experiment_dir, round_name, phase_pattern))
            
            for phase_dir in phase_dirs:
                phase_name = os.path.basename(phase_dir)
                # Extrai o número da fase
                phase_number = re.search(r'^(\d+)', phase_name)
                phase_number = int(phase_number.group(1)) if phase_number else 0
                
                for tenant in tenants:
                    csv_path = os.path.join(phase_dir, tenant, f"{metric_name}.csv")
                    
                    if os.path.exists(csv_path):
                        try:
                            df = pd.read_csv(csv_path)
                            
                            # Adicionar metadados
                            df['tenant'] = tenant
                            df['round'] = round_name
                            df['phase'] = phase_name
                            df['phase_number'] = phase_number
                            
                            # Converter timestamp para datetime
                            df['datetime'] = df['timestamp'].apply(parse_timestamp)
                            
                            all_data.append(df)
                        except Exception as e:
                            print(f"Erro ao carregar {csv_path}: {e}")
    
    if not all_data:
        return pd.DataFrame()
    
    return pd.concat(all_data, ignore_index=True)

## Normalização do Tempo

Para facilitar a análise entre diferentes fases e rounds, vamos converter os timestamps absolutos para tempo decorrido desde o início de cada fase ou do experimento completo.

In [ ]:
def add_elapsed_time(df, group_by=['round', 'phase']):
    """
    Adiciona colunas de tempo decorrido, calculado desde o início de cada grupo.
    
    Args:
        df (DataFrame): DataFrame com dados do experimento
        group_by (list): Colunas para agrupar ao calcular tempo inicial
        
    Returns:
        DataFrame: DataFrame com colunas adicionais de tempo decorrido
    """
    # Cria uma cópia para não modificar o original
    result = df.copy()
    
    # Encontrar o timestamp inicial para cada grupo
    start_times = df.groupby(group_by)['datetime'].min().reset_index()
    
    # Renomear a coluna para facilitar o merge
    start_times = start_times.rename(columns={'datetime': 'start_time'})
    
    # Mesclar com o DataFrame original
    result = pd.merge(result, start_times, on=group_by)
    
    # Calcular tempo decorrido em segundos desde o início de cada grupo
    result['elapsed_seconds'] = (result['datetime'] - result['start_time']).dt.total_seconds()
    
    # Calcular tempo decorrido em minutos (para facilitar a visualização)
    result['elapsed_minutes'] = result['elapsed_seconds'] / 60.0
    
    return result

## Exploração dos Dados Disponíveis

Vamos listar os tenants e métricas disponíveis no nosso conjunto de dados para entender melhor o que temos à disposição.

In [ ]:
# Listar tenants disponíveis
tenants = list_available_tenants(BASE_DATA_DIR)
print(f"Tenants disponíveis: {tenants}")

# Listar métricas disponíveis para um tenant
metrics = list_available_metrics(BASE_DATA_DIR)
print(f"\nMétricas disponíveis: {metrics}")

# Listar rounds disponíveis
rounds = [d for d in os.listdir(BASE_DATA_DIR) if os.path.isdir(os.path.join(BASE_DATA_DIR, d)) and d.startswith("round-")]
print(f"\nRounds disponíveis: {rounds}")

# Listar fases para o primeiro round (como exemplo)
phases = [d for d in os.listdir(os.path.join(BASE_DATA_DIR, rounds[0])) if os.path.isdir(os.path.join(BASE_DATA_DIR, rounds[0], d))]
print(f"\nFases disponíveis (para {rounds[0]}): {phases}")

## Análise de uma Métrica Específica

Vamos carregar os dados de uso de CPU para todos os tenants e visualizar como essa métrica se comporta ao longo das fases do experimento.

In [ ]:
# Carregar dados de CPU para todos os tenants
cpu_data = load_metric_data(BASE_DATA_DIR, "cpu_usage")

# Adicionar tempo decorrido
cpu_data = add_elapsed_time(cpu_data)

# Verificar os dados carregados
print(f"Dimensões do DataFrame: {cpu_data.shape}")
cpu_data.head()

## Visualização do Uso de CPU por Tenant

Vamos criar um gráfico que mostra o uso de CPU para cada tenant ao longo das fases do experimento para o primeiro round.

In [ ]:
# Filtrar para o primeiro round
round1_data = cpu_data[cpu_data['round'] == 'round-1'].copy()

# Criar um gráfico para cada fase
phases = sorted(round1_data['phase'].unique())

plt.figure(figsize=(15, 10))

for i, phase in enumerate(phases):
    plt.subplot(len(phases), 1, i+1)
    
    # Filtrar dados para esta fase
    phase_data = round1_data[round1_data['phase'] == phase]
    
    # Plotar dados para cada tenant
    for tenant in sorted(phase_data['tenant'].unique()):
        tenant_data = phase_data[phase_data['tenant'] == tenant]
        plt.plot(tenant_data['elapsed_minutes'], tenant_data['value'], label=tenant)
    
    plt.title(f"Fase: {phase}")
    plt.xlabel("Tempo Decorrido (minutos)")
    plt.ylabel("Uso de CPU")
    plt.legend()
    plt.grid(True)

plt.tight_layout()
plt.show()

## Análise Comparativa entre Fases

Vamos calcular estatísticas resumidas para comparar o uso de CPU entre diferentes fases para cada tenant.

In [ ]:
# Calcular estatísticas por tenant e fase
summary_stats = cpu_data.groupby(['tenant', 'round', 'phase']).agg({
    'value': ['mean', 'std', 'min', 'max', 'count']
}).reset_index()

# Renomear as colunas para facilitar o acesso
summary_stats.columns = ['tenant', 'round', 'phase', 'mean_cpu', 'std_cpu', 'min_cpu', 'max_cpu', 'count']

# Ordenar por tenant, round e fase
summary_stats = summary_stats.sort_values(['tenant', 'round', 'phase_number'])

# Visualizar as estatísticas
summary_stats

## Visualização Comparativa entre Fases

Vamos criar um gráfico de barras que compara o uso médio de CPU entre as diferentes fases para cada tenant.

In [ ]:
# Criar gráfico comparativo
plt.figure(figsize=(14, 8))

# Agrupar por tenant e fase para o primeiro round
summary_round1 = summary_stats[summary_stats['round'] == 'round-1']

# Normalizar os nomes das fases para exibição no gráfico
summary_round1['phase_name'] = summary_round1['phase'].apply(lambda x: x.split('-')[-1].strip())

# Pivotear os dados para facilitar a plotagem
pivot_data = summary_round1.pivot(index='tenant', columns='phase_name', values='mean_cpu')

# Criar gráfico de barras
ax = pivot_data.plot(kind='bar', figsize=(14, 8))
plt.title('Uso Médio de CPU por Tenant em Diferentes Fases (Round 1)')
plt.xlabel('Tenant')
plt.ylabel('Uso Médio de CPU')
plt.legend(title='Fase')
plt.grid(axis='y')

# Adicionar valores sobre as barras
for container in ax.containers:
    ax.bar_label(container, fmt='%.2f', padding=3)

plt.tight_layout()
plt.show()

## Exportação de Tabelas para Publicações Acadêmicas

Vamos demonstrar como exportar os resultados em formatos adequados para publicações acadêmicas (.csv e .tex).

In [ ]:
# Exportar para CSV
summary_stats.to_csv('cpu_usage_summary.csv', index=False)
print("Tabela exportada como CSV")

# Função para exportar para LaTeX
def export_to_latex(df, caption, label, filename):
    """
    Exporta um DataFrame para uma tabela LaTeX formatada.
    
    Args:
        df (DataFrame): DataFrame a ser exportado
        caption (str): Legenda da tabela
        label (str): Identificador para referência cruzada
        filename (str): Nome do arquivo de saída
    """
    latex_table = df.to_latex(index=False, caption=caption, label=label, float_format="%.2f")
    
    # Adicionar pacotes e formatação adicional
    latex_preamble = """\\documentclass{article}
\\usepackage{booktabs}
\\usepackage{siunitx}
\\usepackage{caption}
\\begin{document}
"""
    latex_end = "\\end{document}"
    
    with open(filename, 'w') as f:
        f.write(latex_preamble)
        f.write(latex_table)
        f.write(latex_end)
    
    print(f"Tabela exportada como LaTeX para {filename}")

# Exportar para LaTeX
export_to_latex(
    summary_stats[summary_stats['round'] == 'round-1'], 
    "Estatísticas de uso de CPU por tenant e fase no Round 1",
    "tab:cpu_usage_round1",
    "cpu_usage_round1.tex"
)

## Conclusão e Próximos Passos

Neste notebook, demonstramos:

1. Como carregar e consolidar dados de diferentes tenants e fases
2. Como normalizar os timestamps para tempo decorrido
3. Como visualizar o uso de CPU ao longo das fases do experimento
4. Como calcular estatísticas comparativas entre fases e tenants
5. Como exportar resultados em formatos adequados para publicações acadêmicas

No próximo notebook, vamos explorar análises mais avançadas, incluindo:
- Correlações entre diferentes métricas
- Detecção de anomalias e padrões
- Análise de impacto do "noisy neighbor" nos demais tenants

Também vamos criar visualizações mais elaboradas que destacam claramente o fenômeno de "noisy neighbor" no ambiente Kubernetes.